In [1]:
import pandas as pd
from pytorch_geometric_dataset import LOL_Dataset
from torch_geometric.transforms import NormalizeFeatures
from train_utils import ConcatNodeCentralities

In [2]:
RAW_FILENAME = "lol_C_M_GM_processed_normalized_D04-19-2024T12_33_31.csv"

lol_df = pd.read_csv(RAW_FILENAME)

In [3]:
lol_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8187 entries, 0 to 8186
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   username                   8187 non-null   object 
 1   tagline                    8186 non-null   object 
 2   tier                       8187 non-null   int64  
 3   level                      8187 non-null   int64  
 4   total_wins                 8187 non-null   int64  
 5   total_losses               8187 non-null   int64  
 6   past_n_wins                8187 non-null   int64  
 7   past_n_losses              8187 non-null   int64  
 8   damage_dealt_to_champions  8187 non-null   float64
 9   damage_taken               8187 non-null   float64
 10  control_wards_placed       8187 non-null   float64
 11  wards_placed               8187 non-null   float64
 12  wards_kills                8187 non-null   float64
 13  kills                      8187 non-null   float

In [4]:
lol_df.head()

,username,tagline,tier,level,total_wins,total_losses,past_n_wins,past_n_losses,damage_dealt_to_champions,damage_taken,...,deaths,assists,kill_participation,creep_score,players_played_with,adc,jungle,mid,support,top
0,yuu13,sus,0,79,146,63,14,6,22431.25,22894.65,...,5.10,7.75,0.616949,164.70,"['Yozu#Lux', 'DARKWINGS#NA3', 'pisslowsupport6...",0,1,0,0,0
1,zz1,NA1,0,139,241,157,12,8,20862.05,22507.35,...,5.20,5.70,0.561837,153.10,"['bradleyyy#0002', 'lyteskin#lyte', 'Neøø#1012...",0,0,1,0,0
2,buenos dias,cat,0,665,318,252,9,11,19728.10,30792.00,...,5.55,7.35,0.586639,171.35,"['Tenacity#NA1', 'Main Sup#XDD', 'zz1#NA1', 'P...",0,1,0,0,0
3,FLY Quad,123,0,53,167,88,10,10,25843.90,21683.30,...,4.30,9.40,0.566972,187.60,"['Cody Sun#COS', 'FLY Busio#000', 'Scibbles#Ma...",0,0,1,0,0
4,Kenvi,NA1,0,305,178,119,12,8,20952.75,30173.00,...,3.70,7.20,0.676647,167.25,"['Lancelot Link#NA1', 'c2 meteos#NA1', 'next t...",0,1,0,0,0


In [5]:
lol_df["tier"].value_counts()

tier
2    7187
1     700
0     300
Name: count, dtype: int64

In [6]:
duplicates = lol_df.duplicated(subset=['username'], keep=False)
duplicate_rows = lol_df[duplicates].sort_values(by="username")
duplicate_rows.head(10)

,username,tagline,tier,level,total_wins,total_losses,past_n_wins,past_n_losses,damage_dealt_to_champions,damage_taken,...,deaths,assists,kill_participation,creep_score,players_played_with,adc,jungle,mid,support,top
3265,888,NA2,2,511,38,34,11,9,19012.95,23117.55,...,5.50,5.45,0.472795,189.40,"['xSantaBearx#NA1', 'The Krow#NA1', 'best lol ...",0,0,1,0,0
4393,888,1995,2,217,244,237,10,10,19249.25,18381.35,...,4.20,4.95,0.351085,191.75,"['UW Scrimm#1v9', 'Cute Timmo#NA1', 'Black Lot...",0,0,0,0,1
3300,ADC,ZYH,1,53,83,47,11,9,19322.15,17944.65,...,5.50,7.05,0.488722,200.60,"['Love Ego#1tap', 'Crucile#NA1', 'ttv ssyeu#ss...",1,0,0,0,0
7382,ADC,MUNCH,2,670,340,353,8,12,20643.00,20190.95,...,6.85,5.20,0.515275,174.70,"['Kakoos#NA1', 'Derek#VNM1', 'kevinnguyensocal...",1,0,0,0,0
4291,Adam,GB2,2,50,123,87,9,11,16547.05,16492.40,...,4.95,5.80,0.443763,182.25,"['FR1DGE#USA', 'Bouj#NA1', 'Bungo BALLS#BOOM',...",1,0,0,0,0
6131,Adam,NA02,2,78,31,19,10,9,9998.15,21992.35,...,5.45,11.25,0.494792,31.40,"['SIEGE MACHINE#NA1', 'FufuSauceGraine#Afrik',...",0,0,0,1,0
1554,Asleep,THSMS,2,63,129,101,16,4,8977.10,12579.95,...,3.20,12.30,0.647321,33.40,"['Wizard101 BF#Zach', 'Mori#88888', 'Nimb#CAT'...",0,0,0,1,0
4251,Asleep,Pengu,2,754,38,30,11,9,12483.30,16710.30,...,4.40,12.50,0.649378,30.90,"['Akadian Pasha#NA1', '邪悪な#Dev', 'repiv#0000',...",0,0,0,1,0
1176,Asuna,5103,0,74,304,250,14,6,12597.00,16788.90,...,3.65,13.75,0.671518,32.30,"['12 Pouces Mou#NA1', 'lyteskin#lyte', 'Sharpe...",0,0,0,1,0
8034,Asuna,1Yuki,1,707,130,106,13,7,14609.45,21501.35,...,3.45,4.70,0.385417,156.40,"['NoPause#클로버', 'Super Big Ego#NA1', 'Celerity...",0,0,0,0,1


In [15]:
lol_graph = LOL_Dataset(
    root="PTG_data",
    transform=NormalizeFeatures(),
    pre_transform=ConcatNodeCentralities()
)

In [16]:
print()
print(f'lol_graph: {lol_graph}:')
print('======================')
print(f'Number of graphs: {len(lol_graph)}')
print(f'Number of features: {lol_graph.num_features}')
print(f'Number of classes: {lol_graph.num_classes}')
graph1 = lol_graph[0]  # Get the first graph object.

print()
print(graph1)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {graph1.num_nodes}')
print(f'Number of edges: {graph1.num_edges}')
print(f'Average node degree: {graph1.num_edges / graph1.num_nodes:.2f}')
print(f'Has isolated nodes: {graph1.has_isolated_nodes()}')
print(f'Has self-loops: {graph1.has_self_loops()}')
print(f'Is undirected: {graph1.is_undirected()}')
print(graph1.x[0])


lol_graph: LOL_Dataset():
Number of graphs: 1
Number of features: 24
Number of classes: 3

Data(x=[8187, 24], edge_index=[2, 1625502], y=[8187])
Number of nodes: 8187
Number of edges: 1625502
Average node degree: 198.55
Has isolated nodes: False
Has self-loops: False
Is undirected: True
tensor([1.6547e-03, 3.0580e-03, 1.3195e-03, 2.9323e-04, 1.2567e-04, 4.6982e-01,
        4.7953e-01, 2.4087e-05, 1.6128e-04, 4.9221e-05, 2.1887e-04, 1.0682e-04,
        1.6232e-04, 1.2922e-05, 3.4496e-03, 0.0000e+00, 2.0945e-05, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 4.6561e-07, 9.1311e-06, 3.9984e-02, 2.2262e-07])


In [9]:
TEST_FILENAME = "centralities_df.csv"

test_df = pd.read_csv(TEST_FILENAME)

In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8187 entries, 0 to 8186
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                8187 non-null   int64  
 1   degree_centralities       8187 non-null   float64
 2   closeness_centralities    8187 non-null   float64
 3   betweenness_centralities  8187 non-null   float64
 4   eigen_centralities        8187 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 319.9 KB


In [11]:
test_df.head()

,Unnamed: 0,degree_centralities,closeness_centralities,betweenness_centralities,eigen_centralities
0,0,0.022230,0.435959,1909.002528,0.010629
1,1,0.039209,0.458240,6732.671144,0.019363
2,2,0.030903,0.450077,3930.245067,0.015112
3,3,0.018932,0.436098,1829.389557,0.009329
4,4,0.018566,0.438364,1121.250226,0.010399


In [12]:
TEST_FILENAME2 = "concat_features_df.csv"

test2_df = pd.read_csv(TEST_FILENAME2)
test2_df.info()
test2_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8187 entries, 0 to 8186
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8187 non-null   int64  
 1   0           8187 non-null   float64
 2   1           8187 non-null   float64
 3   2           8187 non-null   float64
 4   3           8187 non-null   float64
 5   4           8187 non-null   float64
 6   5           8187 non-null   float64
 7   6           8187 non-null   float64
 8   7           8187 non-null   float64
 9   8           8187 non-null   float64
 10  9           8187 non-null   float64
 11  10          8187 non-null   float64
 12  11          8187 non-null   float64
 13  12          8187 non-null   float64
 14  13          8187 non-null   float64
 15  14          8187 non-null   float64
 16  15          8187 non-null   float64
 17  16          8187 non-null   float64
 18  17          8187 non-null   float64
 19  18          8187 non-null  

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,0,79.0,146.0,63.0,14.0,6.0,22431.25,22894.65,1.15,7.70,...,164.70,0.0,1.0,0.0,0.0,0.0,0.022230,0.435959,1909.0026,0.010629
1,1,139.0,241.0,157.0,12.0,8.0,20862.05,22507.35,1.85,10.10,...,153.10,0.0,0.0,1.0,0.0,0.0,0.039209,0.458240,6732.6714,0.019363
2,2,665.0,318.0,252.0,9.0,11.0,19728.10,30792.00,4.75,7.95,...,171.35,0.0,1.0,0.0,0.0,0.0,0.030903,0.450077,3930.2450,0.015112
3,3,53.0,167.0,88.0,10.0,10.0,25843.90,21683.30,6.15,14.05,...,187.60,0.0,0.0,1.0,0.0,0.0,0.018932,0.436098,1829.3895,0.009329
4,4,305.0,178.0,119.0,12.0,8.0,20952.75,30173.00,3.80,9.35,...,167.25,0.0,1.0,0.0,0.0,0.0,0.018566,0.438364,1121.2502,0.010399
